In [1]:
import pandas as pd

In [2]:
t1=pd.read_csv("tmdb_5000_credits.csv")
t2=pd.read_csv("tmdb_5000_movies.csv")

In [3]:
t=pd.merge(t1, t2, left_on='movie_id', right_on='id', how = 'inner')

In [4]:
t.drop(["id","movie_id","original_language","title_y","status","homepage","original_title","vote_average","vote_count","release_date","revenue","budget","popularity","runtime"],axis=1,inplace=True)

In [5]:
t.isnull().sum()

title_x                   0
cast                      0
crew                      0
genres                    0
keywords                  0
overview                  3
production_companies      0
production_countries      0
spoken_languages          0
tagline                 844
dtype: int64

In [6]:
t.dropna(inplace=True)
t.reset_index(inplace=True,drop=True)

In [7]:
t.duplicated().sum()

0

In [8]:
import re
import ast
def extract_acname(k):
    c=0
    li=[]
    for i in ast.literal_eval(k):
        li.append(i["name"].replace(" ",""))
        if c==3:
            break
        c=c+1
    return li

In [9]:
t["cast"]=t["cast"].apply(extract_acname)

In [10]:
def extract_diname(k):
    li=[]
    for i in ast.literal_eval(k):
            j=dict(i)
            if "Directing" in j["department"]:
                li.append(j["name"].replace(" ",""))
    return li
            
        

In [11]:
t["crew"]=t["crew"].apply(extract_diname)

In [12]:
def extract_gename(k):
    li=[]
    for i in ast.literal_eval(k):
        li.append(i["name"].replace(" ",""))
    return li
    

In [13]:
t["genres"]=t["genres"].apply(extract_gename)

In [14]:
def extract_proname(k):
    li=[]
    for i in ast.literal_eval(k):
        li.append(i["name"].replace(" ",""))
    return li

In [15]:
t["production_companies"]=t["production_companies"].apply(extract_proname)

In [16]:
def extract_proconame(k):
    li=[]
    for i in ast.literal_eval(k):
        li.append(i["iso_3166_1"])
    return li

In [17]:
t["production_countries"]=t["production_countries"].apply(extract_proconame)

In [18]:
def spoken(k):
    li=[]
    for i in ast.literal_eval(k):
        li.append(i["iso_639_1"])
    return li

In [19]:
t["spoken_languages"]=t["spoken_languages"].apply(spoken)

In [20]:
def spoken(k):
    li=[]
    for i in ast.literal_eval(k):
        li.append(i["name"])
    return li

In [21]:
t["keywords"]=t["keywords"].apply(spoken)

In [22]:
def concat_rec(yu):
    return ' '.join(yu)
t["cast"]=t["cast"].apply(concat_rec)
t["crew"]=t["crew"].apply(concat_rec)
t["genres"]=t["genres"].apply(concat_rec)
t["keywords"]=t["keywords"].apply(concat_rec)
t["production_companies"]=t["production_companies"].apply(concat_rec)
t["production_countries"]=t["production_countries"].apply(concat_rec)
t["spoken_languages"]=t["spoken_languages"].apply(concat_rec)

In [23]:
def wdfsdvc(l):
    return ' '.join(map(str, l))
tt=pd.DataFrame(t.apply(wdfsdvc, axis=1),columns=["new"])
t.drop(["cast","crew","genres","keywords","overview","production_companies","production_countries","spoken_languages","tagline"],axis=1,inplace=True)

In [24]:
tt.head()

,new
0,Avatar SamWorthington ZoeSaldana SigourneyWeav...
1,Pirates of the Caribbean: At World's End Johnn...
2,Spectre DanielCraig ChristophWaltz LéaSeydoux ...
3,The Dark Knight Rises ChristianBale MichaelCai...
4,John Carter TaylorKitsch LynnCollins SamanthaM...


In [25]:
t.head()

,title_x
0,Avatar
1,Pirates of the Caribbean: At World's End
2,Spectre
3,The Dark Knight Rises
4,John Carter


In [26]:
from nltk.stem import WordNetLemmatizer as WNL
from nltk import word_tokenize
import nltk
from nltk.corpus import wordnet 
from autocorrect import Speller

In [27]:
get_wordnet_pos=lambda treebank_tag: wordnet.ADJ if treebank_tag[0]=="J" else wordnet.VERB if treebank_tag[0]=="V" else wordnet.ADV if treebank_tag[0]=="R" else wordnet.NOUN 

In [28]:
wnl = WNL()

def lemmatize_text(text):
    return " ".join([wnl.lemmatize(t,pos=get_wordnet_pos(nltk.pos_tag([t])[0][1])) for t in word_tokenize(text)])
tt["new"]=tt["new"].apply(lemmatize_text)

In [29]:
from sklearn.feature_extraction.text import CountVectorizer

In [30]:
cv=CountVectorizer(binary=True,max_features=5000,stop_words ="english")

In [31]:
rec=cv.fit_transform(tt["new"]).toarray()

In [32]:
from sklearn.metrics.pairwise import cosine_similarity

In [33]:
cs=pd.DataFrame(cosine_similarity(rec))

In [34]:
cs.set_index(t.index,inplace=True)

In [35]:
def recommend(y):
    movie=[]
    index_movie=t[(t.title_x==y)].index[0]
    for i in sorted(list(enumerate(cs[index_movie])),reverse=True,key=lambda x:x[1])[1:7]:
        movie.append(i[0])
    return list(t.loc[movie,:].title_x.values)

In [36]:
recommend("The Dark Knight Rises")

['The Dark Knight',
 'Batman Begins',
 'Batman',
 'Batman Returns',
 'Batman Forever',
 'The Prestige']